In [1]:
import numpy as np
import sys
from pathlib import Path
import glob
import os
from suite2p import run_s2p
ops = run_s2p.default_ops() # populates ops with the default options
# option to import from github folder
# sys.path.insert(0, 'C:/Users/trose/Documents/GitHub/suite2p')

In [2]:
# own imports
from ScanImageTiffReader import ScanImageTiffReader
from helpers import parse_SI_header as pSI #own
from shutil import copy, rmtree
from tqdm import tqdm
#import pandas as pd

### File handling (PC / Mac)

In [3]:
exp = ['62282', '62283', '62284', '62285', '62286', '62287', '62288', '62289', '62290', '62291', '62292', '62293', '62304', '62305', '62306', '62307', '62308', '62309', '62359', '62360', '62361', '62362', '62363', '62364']

In [4]:
exp = ['62335']

In [5]:
if sys.platform == "win32":
    main_root = 'I:/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/' #location of original data
    ftemp = 'C:/temp/trose/suite2ptemp'        #fast disk
    if len(exp)>1:
        froot = 'C:/temp/trose/s2p_tiff/'      #temp drive for concatenating exp tiffs
        fsave =  froot 
    else:
        froot = main_root                      #No concatenation of experiments -> can work on NW drive
        fsave = froot
elif sys.platform == "darwin":
    main_root = '/Volumes/archive_bonhoeffer_group$/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/' #location of original data
    ftemp = '/Users/trose/Data/temp/'          #fast disk
    if len(exp)>1:
        froot = '/Users/trose/Data/s2p_tiff/'  #temp drive for concatenating exp tiffs
        fsave =  froot 
    else:
        froot = main_root                      #No concatenation of experiments -> can work on NW drive
        fsave = froot
else: 
    print('Unkknown Platform')

fsave_tmp = froot + '/suite2p_exp' + exp[0]
ftemp_tmp = ftemp + '/suite2p'

Path(fsave_tmp).mkdir(parents=True, exist_ok=True)
Path(ftemp_tmp).mkdir(parents=True, exist_ok=True)

In [6]:
for ids,val in enumerate(exp):
    print(val)

62335


In [7]:
# Copy all tiffs to a temp folder for concatenated handling
if len(exp)>1:
    files_fullpath = []
    readfiles = 2
    for val in exp:
        files = list(Path(main_root).rglob('exp'+val+'*.tif')) #recursive search over main_root
        files = sorted(files)
        files = files[0:readfiles]
        for n,f in enumerate(files):
            copy(files[n], froot)
            targetstring = [str(files[n]), str(froot)]
            print('copied {} to {}'.format(*targetstring))

copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62335_DL_002_001.tif to C:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62335_DL_002_002.tif to C:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_001.tif to C:/temp/trose/s2p_tiff/
copied I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\exp62336_DL_003_002.tif to C:/temp/trose/s2p_tiff/


In [42]:
main_root

'I:/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/'

In [13]:
files_fullpath = []
files_name = []
readfiles = 5

for val in exp:
    #files = list(Path(froot).glob('exp'+val+'*.tif'))
    files = list(Path(froot).rglob('exp'+val+'*.tif')) #recursive
    files = files[0:readfiles]
    #files = glob.glob(os.path.join(froot, 'exp'+val+'*.tif'))
    for n,f in enumerate(files):
        #files[n] = os.path.basename(f)
        files_fullpath.append(str(files[n]))
        files_name.append(f.name)

    
files_fullpath = sorted(files_fullpath)
files_name = sorted(files_name)

In [14]:
os.path.dirname(files[0])

'I:\\David Laubender\\Data\\imaging data\\DL_191106_2\\ImagingData\\2019-12-14'

In [15]:
with ScanImageTiffReader(files_fullpath[0]) as reader:
    header = (reader.description(0))
    mov_dim = (reader.shape())
    
level = pSI.parse_SI_header_level(header)
zoom = pSI.parse_SI_header_zoom(header)
framerate = pSI.parse_SI_header_FrameRate(header)
channels = pSI.parse_SI_header_Channels(header)
volumes = pSI.parse_SI_header_Volumes(header)
frames = pSI.parse_SI_header_Frames(header)
frames_per_file = pSI.parse_SI_header_FramesPerFile(header)

# account for multilevel acq where frames is 1
if frames < volumes:
    frames = volumes

In [16]:
bad_frames = np.linspace(0, 49, 50)
np.save(fsave_tmp + '/bad_frames.npy', bad_frames)
np.save(ftemp_tmp + '/bad_frames.npy', bad_frames)
#np.save(ftemp + '/bad_frames.npy', bad_frames)

In [17]:
db = {
      'h5py': [], # a single h5 file path
      'h5py_key': 'data', # file paths
      'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
      'data_path': [froot],  # a list of folders with tiffs
                             # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
      'delete_bin': False,
      'save_path0': fsave,
      'fast_disk': ftemp,
      'subfolders': [], # choose subfolders of 'data_path' to look in (optional)

      # main settings
      'nplanes': level, # each tiff has these many planes in sequence
      'nchannels': channels, # each tiff has these many channels per plane
      'functional_chan': 1, # this channel is used to extract functional ROIs (1-based)
      'tau':  1., # this is the main parameter for deconvolution
      'fs': framerate / level,  # sampling rate (PER PLANE - e.g. if you have 12 planes then this should be around 2.5)
      'preclassify': 0, # apply classifier before signal extraction with probability 0.5 (turn off with value 0)
      'frames_include:': -1,  #default: -1) if greater than zero, only frames_include frames are processed. useful for testing parameters on a subset of data.
      # output settings
      'save_mat': True, # whether to save output as matlab files
      'combined': True, # combine multiple planes into a single result /single canvas for GUI

      'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
      'num_workers_roi': 0, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
      'force_sktiff': False,
      # bidirectional phase offset
      'do_bidiphase': True,
      'bidiphase': 0,

      # registration settings
      'do_registration': 2, # whether to register data (2 forces re-registration)
      'two_step_registration:': True, #default: False) whether or not to run registration twice (for low SNR data). keep_movie_raw must be True for this to work.
      'keep_movie_raw': True,
      'nimg_init': 400, # subsampled frames for finding reference image
      'batch_size': 800, # number of frames per batch
      'maxregshift': 0.05, # max allowed registration shift, as a fraction of frame max(width and height)
      'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
      'reg_tif': True, # whether to save registered tiffs
      'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
      'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
      'smooth_sigma': 1.5, # ~1 good for 2P recordings, recommend >5 for 1P recordings
      'smooth_sigma_time': 2, # default: 0) standard deviation in time frames of the gaussian used to smooth the data before phase correlation is computed. Might need this to be set to 1 or 2 for low SNR data.
      'th_badframes': 2, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
      'pad_fft': False,

      # non rigid registration settings
      'nonrigid': True, # whether to use nonrigid registration
      'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
      'snr_thresh': 1.5, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
      'maxregshiftNR': 8, # maximum pixel shift allowed for nonrigid, relative to rigid

      # cell detection settings
      'roidetect': True, # whether or not to run ROI extraction
      'spatial_scale': 0, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
      'diameter': [9,12], #this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
      'connected': False, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
      'nbinned': 5000, # max number of binned frames for cell detection
      'max_iterations': 50, # maximum number of iterations to do cell detection
      'threshold_scaling': 2.0, # adjust the automatically determined threshold by this scalar multiplier
      'max_overlap': 0.9, # cells with more overlap than this get removed during triage, before refinement
      'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)
      'smooth_masks': True, # default: True) whether to smooth masks in final pass of cell detection. This is useful especially if you are in a high noise regime.

      # ROI extraction parameters
      'sparse_mode': False, #default: False) whether or not to use sparse_mode cell detection
      'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
      'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
      'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)

      # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
      'chan2_thres': 0.65, # minimum for detection of brightness on channel 2

      # deconvolution settings
      'baseline': 'maximin', # baselining mode (can also choose 'prctile')
      'win_baseline': 60., # window for maximin
      'sig_baseline': 10., # smoothing constant for gaussian filter
      'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
      'neucoeff': .7,  # neuropil coefficient


      # List of tiffs to be loaded
      'tiff_list': files_name # list of tiffs in folder * data_path *!
}


In [18]:
opsEnd = run_s2p.run_s2p(ops=ops, db=db)

{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['I:/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/'], 'delete_bin': False, 'save_path0': 'I:/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/', 'fast_disk': 'C:/temp/trose/suite2ptemp', 'subfolders': [], 'nplanes': 4, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 7.5, 'preclassify': 0, 'frames_include:': -1, 'save_mat': True, 'combined': True, 'num_workers': 0, 'num_workers_roi': 0, 'force_sktiff': False, 'do_bidiphase': True, 'bidiphase': 0, 'do_registration': 2, 'two_step_registration:': True, 'keep_movie_raw': True, 'nimg_init': 400, 'batch_size': 800, 'maxregshift': 0.05, 'align_by_chan': 1, 'reg_tif': True, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma': 1.5, 'smooth_sigma_time': 2, 'th_badframes': 2, 'pad_fft': False, 'nonrigid': True, 'block_size': [128, 128], 'snr_thresh': 1.5, 'maxregshiftNR': 8, 'roidetect': True, 'spatial_scale': 0, 'diam

C:\Miniconda3\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 6.71 sec.
Extracted fluorescence from 178 ROIs in 600 frames, 3.40 sec.
----------- Total 59.93 sec.
----------- SPIKE DECONVOLUTION
----------- Total 28.92 sec.
Plane 0 processed in 256.70 sec (can open in GUI).
total = 282.79 sec.
>>>>>>>>>>>>>>>>>>>>> PLANE 1 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 600 frames
NOTE: estimated bidiphase offset from data: 0 pixels
Reference frame, 99.93 sec.
600/600 frames, 61.33 sec.
bad frames file path: I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\bad_frames.npy
----------- Total 162.55 sec
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 08
Binned movie [62,502,489], 1.18 sec.
ROIs: 145, cost: 0.5350, time: 17.1941
ROIs: 175, cost: 0.5278, time: 21.1711
ROIs: 179, cost: 0.5265, time: 23.7291
Binning movie in chunks of length 08
Binned movie [62,502,489], 1.16 sec.
ROIs: 179, cost: 0.5543, time: 26.6381
ROIs: 179, cost: 0.5407, time: 28.4971
ROIs: 179, cost: 0.540

C:\Miniconda3\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 6.45 sec.
Extracted fluorescence from 178 ROIs in 600 frames, 3.46 sec.
----------- Total 61.22 sec.
----------- SPIKE DECONVOLUTION
----------- Total 30.74 sec.
Plane 1 processed in 260.02 sec (can open in GUI).
total = 542.81 sec.
>>>>>>>>>>>>>>>>>>>>> PLANE 2 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 600 frames
NOTE: estimated bidiphase offset from data: 1 pixels
Reference frame, 102.69 sec.
600/600 frames, 68.39 sec.
bad frames file path: I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\bad_frames.npy
----------- Total 172.31 sec
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 08
Binned movie [62,502,487], 1.17 sec.
ROIs: 127, cost: 0.5354, time: 17.9820
ROIs: 150, cost: 0.5286, time: 21.4710
ROIs: 152, cost: 0.5276, time: 23.8710
Binning movie in chunks of length 08
Binned movie [62,502,487], 1.15 sec.
ROIs: 152, cost: 0.5584, time: 26.3700
ROIs: 152, cost: 0.5449, time: 28.2550
ROIs: 152, cost: 0.54

C:\Miniconda3\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 6.41 sec.
Extracted fluorescence from 152 ROIs in 600 frames, 3.62 sec.
----------- Total 61.03 sec.
----------- SPIKE DECONVOLUTION
----------- Total 32.67 sec.
Plane 2 processed in 270.61 sec (can open in GUI).
total = 813.42 sec.
>>>>>>>>>>>>>>>>>>>>> PLANE 3 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 600 frames
NOTE: estimated bidiphase offset from data: 1 pixels
Reference frame, 108.36 sec.
600/600 frames, 66.74 sec.
bad frames file path: I:\David Laubender\Data\imaging data\DL_191106_2\ImagingData\2019-12-14\bad_frames.npy
----------- Total 176.65 sec
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 08
Binned movie [75,502,483], 1.51 sec.
ROIs: 141, cost: 0.5336, time: 20.3660
ROIs: 162, cost: 0.5284, time: 23.8330
ROIs: 164, cost: 0.5277, time: 26.4800
Binning movie in chunks of length 08
Binned movie [75,502,483], 1.35 sec.
ROIs: 164, cost: 0.5537, time: 29.4800
ROIs: 164, cost: 0.5438, time: 31.5360
ROIs: 164, cost: 0.54

C:\Miniconda3\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 5.94 sec.
Extracted fluorescence from 162 ROIs in 600 frames, 3.69 sec.
----------- Total 63.98 sec.
----------- SPIKE DECONVOLUTION
----------- Total 33.71 sec.
Plane 3 processed in 279.52 sec (can open in GUI).
total = 1092.95 sec.
TOTAL RUNTIME 1116.48 sec


### convert notebook to *.py

In [10]:
!jupyter nbconvert --to script suite2p_axon_workflow.ipynb

[NbConvertApp] Converting notebook suite2p_axon_workflow.ipynb to script
[NbConvertApp] Writing 9850 bytes to suite2p_axon_workflow.py


# Trash collector

In [ ]:
stat = np.load(ftemp +'/std_ops/ops_dendrite.npy', allow_pickle=True)

In [ ]:
# set your options for running
ops = run_s2p.default_ops() # populates ops with the default options

In [ ]:
opspath = 'C:/temp/trose/David Laubender/Data/imaging data/DL_191106_2/ImagingData/2019-12-14/suite2p/suite2p/plane0/ops.npy'

stato = np.load(opspath, allow_pickle=True)

In [ ]:
print(stat.conj().transpose())
